# Portfolio Analysis

In this project, we seek to run through some common portfolio construction and asset allocation strategies to gain some insights into the usefulness of standard measures like sharpe ratio. The end goal is to test such strategies to a sufficient level of rigour that we can convincingly employ them into our portfolio rebalancing pipeline. I will start off by testing my own portfolio and the procedures should be generalizable. Let's get started!

## 0. Project Setup

##### Assets

1. Sea Limited (NYSE:'**SE**')
2. Bilibili Inc (NASDAQ:'**BILI**')
3. Pinduoduo Inc (NASDAQ:'**PDD**')
4. NVIDIA Corporation (NASDAQ:'**NVDA**') 
5. Unity Software Inc (NYSE:'**U**')
6. Peloton Inc (NASDAQ:'**PTON**')
7. Alphabet Inc (NASDAQ:'**GOOG**)
8. Amazon.com Inc (NASDAQ:'**AMZN**)
9. Microsoft Corporation (NASDAQ:'**MSFT**')
10. Facebook Inc (NASDAQ:'**FB**')
11. Intel Corporation (NASDAQ: '**INTC**')
12. Roku Inc (NASDAQ:'**ROKU**')
13. Roblox Corp (NYSE:'**RBLX**')
14. Blackrock Inc (NYSE:'**BLK**')
15. Berkshire Hathaway Inc (NYSE:'**BRK-B**')
16. Autodesk, Inc. (NASDAQ:'**ADSK**')
17. Draftkings Inc (NASDAQ:'**DKNG**')
18. Clearpoint Neuro Inc (NASDAQ:'**CLPT**')

##### Data

Daily adjusted closing prices from yahoo finance API 

##### Time period

1 year, 3 year, 5 year & 10 year.

In [1]:
# Load the required packages 
# Computation
import numpy as np 
from scipy import fftpack
# Plotting
import matplotlib.pyplot as plt
import mplcursors
import matplotlib.ticker as mtick
import seaborn as sns
# Data analysis
import pandas as pd
from sklearn import preprocessing
# Data source
import yfinance as yf

## 1. Data Analysis

In [2]:
# shortlisted stocks for portfolio analysis
s_list = 'SE BILI PDD NVDA PTON U INTC GOOG AMZN ROKU MSFT FB BLK BRK-B RBLX ADSK DKNG CLPT'
df_1y = yf.download(tickers = s_list, period = '1y', interval = '1d', group_by = 'ticker')
df_3y = yf.download(tickers = s_list, period = '3y', interval = '1d', group_by = 'ticker')
df_5y = yf.download(tickers = s_list, period = '5y', interval = '1d', group_by = 'ticker')
df_10y = yf.download(tickers = s_list, period = '10y', interval = '1d', group_by = 'ticker')

# Benchmark - S&P500
bench = '^GSPC'
bench_1y = yf.download(tickers = bench, period = '1y', interval = '1d')
bench_3y = yf.download(tickers = bench, period = '3y', interval = '1d')
bench_5y = yf.download(tickers = bench, period = '5y', interval = '1d')
bench_10y = yf.download(tickers = bench, period = '10y', interval = '1d')

[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [35]:
# select column for adjusted close prices
df_close_1y = df_1y.xs('Adj Close', level = 1, axis = 1)
df_close_3y = df_3y.xs('Adj Close', level = 1, axis = 1)
df_close_5y = df_5y.xs('Adj Close', level = 1, axis = 1)
df_close_10y = df_10y.xs('Adj Close', level = 1, axis = 1)
bench_close_1y = pd.DataFrame(data = bench_1y['Adj Close'], columns = ['Adj Close'])
bench_close_3y = pd.DataFrame(data = bench_3y['Adj Close'], columns = ['Adj Close'])
bench_close_5y = pd.DataFrame(data = bench_5y['Adj Close'], columns = ['Adj Close'])
bench_close_10y = pd.DataFrame(data = bench_10y['Adj Close'], columns = ['Adj Close'])

# adding column name to benchmark table
bench_close_1y = bench_close_1y.rename(columns = {'Adj Close':'S&P500'})
bench_close_3y = bench_close_3y.rename(columns = {'Adj Close':'S&P500'})
bench_close_5y = bench_close_5y.rename(columns = {'Adj Close':'S&P500'})
bench_close_10y = bench_close_10y.rename(columns = {'Adj Close':'S&P500'})
 
# reordering column names in portfolio 
s_order = ['SE', 'BILI', 'NVDA', 'PDD', 'U', 'FB', 'GOOG', 'AMZN', 'MSFT', 'INTC', 'ROKU', 'PTON', 'RBLX',  'BLK', 'BRK-B', 'ADSK', 'DKNG', 'CLPT']
df_close_1y = df_close_1y[s_order]
df_close_3y = df_close_3y[s_order]
df_close_5y = df_close_5y[s_order]
df_close_10y = df_close_10y[s_order]

# appending benchmark prices to portfolio dataframe 
df_close_1y = pd.concat([df_close_1y,bench_close_1y], axis = 1)
df_close_3y = pd.concat([df_close_3y,bench_close_3y], axis = 1)
df_close_5y = pd.concat([df_close_5y,bench_close_5y], axis = 1)
df_close_10y = pd.concat([df_close_10y,bench_close_10y], axis = 1)

# drop row if all values are NaN
df_close_1y.dropna(axis = 0, how = 'all', inplace = True)
df_close_3y.dropna(axis = 0, how = 'all', inplace = True)
df_close_5y.dropna(axis = 0, how = 'all', inplace = True)
df_close_10y.dropna(axis = 0, how = 'all', inplace = True)

# check the last 5 trading days
df_close_1y.tail(5)

,SE,BILI,NVDA,PDD,U,FB,GOOG,AMZN,MSFT,INTC,ROKU,PTON,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
Date,,,,,,,,,,,,,,,,,,,
2021-08-24,312.000000,78.410004,217.929993,99.120003,125.110001,365.510010,2847.969971,3305.780029,302.619995,53.810001,356.820007,113.709999,89.260002,929.190002,286.019989,341.070007,56.470001,17.930000,4486.229980
2021-08-25,320.089996,79.620003,222.130005,97.839996,124.620003,368.390015,2859.000000,3299.179932,302.010010,53.810001,353.470001,116.250000,90.339996,939.520020,287.299988,342.269989,60.110001,17.870001,4496.189941
2021-08-26,321.769989,77.489998,220.679993,95.860001,120.059998,364.380005,2842.459961,3316.000000,299.089996,53.130001,352.000000,114.089996,85.809998,935.880005,285.269989,310.190002,57.860001,17.530001,4470.000000
2021-08-27,321.029999,75.330002,226.360001,94.629997,123.510002,372.630005,2891.010010,3349.629883,299.720001,53.889999,357.029999,104.339996,85.400002,954.940002,286.600006,315.640015,60.009998,18.469999,4509.370117
2021-08-30,331.820007,74.129997,226.880005,94.980003,126.339996,380.660004,2909.389893,3421.570068,303.589996,53.939999,355.899994,101.480003,81.870003,948.080017,285.660004,313.390015,59.250000,18.209999,4528.790039


In [4]:
# normalize the prices 
n = len(df_close_1y.columns)
df_close_1y_norm = df_close_1y.copy()
df_close_3y_norm = df_close_3y.copy()
df_close_5y_norm = df_close_5y.copy()
df_close_10y_norm = df_close_10y.copy()

for i in range(n): 
    a = df_close_1y.iloc[:,i].first_valid_index()
    df_close_1y_start = df_close_1y.iloc[:,i].loc[a]
    df_close_1y_norm.iloc[:,i] = (df_close_1y.iloc[:,i] - df_close_1y_start)/df_close_1y_start
    
    b = df_close_3y.iloc[:,i].first_valid_index()
    df_close_3y_start = df_close_3y.iloc[:,i].loc[b]
    df_close_3y_norm.iloc[:,i] = (df_close_3y.iloc[:,i] - df_close_3y_start)/df_close_3y_start
    
    c = df_close_5y.iloc[:,i].first_valid_index()
    df_close_5y_start = df_close_5y.iloc[:,i].loc[c]
    df_close_5y_norm.iloc[:,i] = (df_close_5y.iloc[:,i] - df_close_5y_start)/df_close_5y_start
    
    d = df_close_10y.iloc[:,i].first_valid_index()
    df_close_10y_start = df_close_10y.iloc[:,i].loc[d]
    df_close_10y_norm.iloc[:,i] = (df_close_10y.iloc[:,i] - df_close_10y_start)/df_close_10y_start

In [5]:
# plotting out the prices
%matplotlib widget

# plot configurations
sns.set(style="darkgrid", font_scale=0.8)
palette = sns.color_palette("hls", 19)
fig, ax = plt.subplots(figsize=(8, 4))

# plotting out the figure
plot1 = sns.lineplot(ax=ax, data = df_close_1y_norm, dashes = False, palette=palette)
plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0., fontsize = 8)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title('1 Year Returns')

# make it interactive
cursor = mplcursors.cursor(plot1, hover=True)
@cursor.connect("add")
def on_add(sel):
    sel.annotation.set(text=tt[sel.target.index])
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# convert the price into rate of return
df_return_1y = df_close_1y.pct_change()
df_return_3y = df_close_3y.pct_change()
df_return_5y = df_close_5y.pct_change()
df_return_10y = df_close_10y.pct_change()

# check the descriptive statistics of 1 year rate of return
df_return_1y.describe()

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
count,251.000000,251.000000,251.000000,251.000000,238.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,120.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.003724,0.002928,0.001329,0.002475,0.003461,0.002074,0.002438,0.000131,0.001339,0.001244,0.000549,0.003607,0.002364,0.002065,0.001137,0.001182,0.002807,0.006964,0.001072
std,0.035502,0.047968,0.047045,0.026942,0.042007,0.043541,0.016613,0.018210,0.015371,0.020259,0.020807,0.038864,0.045345,0.015248,0.010987,0.020521,0.039047,0.051103,0.009459
min,-0.103195,-0.170397,-0.122561,-0.092775,-0.141312,-0.202853,-0.054630,-0.075649,-0.061947,-0.063099,-0.105751,-0.124132,-0.123584,-0.046489,-0.029872,-0.093727,-0.084906,-0.165829,-0.035288
25%,-0.016962,-0.027267,-0.025790,-0.011649,-0.023977,-0.021614,-0.005528,-0.009322,-0.005965,-0.011444,-0.010246,-0.018891,-0.027414,-0.007038,-0.005600,-0.008984,-0.023375,-0.022805,-0.003830
50%,0.005249,0.001162,-0.002952,0.003161,0.000207,0.002646,0.002515,0.001084,0.000965,-0.000059,0.000471,0.000341,0.000093,0.002676,0.001227,0.003090,0.000166,0.000423,0.001257
75%,0.026284,0.031746,0.021318,0.019153,0.028716,0.028121,0.010333,0.010288,0.010617,0.014286,0.012008,0.023022,0.029257,0.011698,0.007013,0.014841,0.025298,0.034452,0.007386
max,0.108809,0.221735,0.222496,0.080333,0.162555,0.144746,0.073961,0.063230,0.048249,0.083227,0.069684,0.176669,0.213281,0.040412,0.060570,0.047380,0.172697,0.164811,0.023791


### Rate of Return

In [7]:
# compute the annualized average rate of return across a 1 year, 3 year, 5 year & 10 year horizon.
nday = 252 # number of trading days
df_return_1y_mean = pd.DataFrame(data = df_return_1y.mean(axis = 0), columns = ['1Y Annual Return']) * nday
df_return_3y_mean = pd.DataFrame(data = df_return_3y.mean(axis = 0), columns = ['3Y Annual Return']) * nday
df_return_5y_mean = pd.DataFrame(data = df_return_5y.mean(axis = 0), columns = ['5Y Annual Return']) * nday
df_return_10y_mean = pd.DataFrame(data = df_return_10y.mean(axis = 0), columns = ['10Y Annual Return']) * nday
data_frames = [df_return_1y_mean, df_return_3y_mean, df_return_5y_mean, df_return_10y_mean]
df_return_merge = pd.concat(data_frames, axis = 1)

#df_return_10y_mean.plot.barh()
df_return_merge

,1Y Annual Return,3Y Annual Return,5Y Annual Return,10Y Annual Return
SE,0.938347,1.227915,0.956266,0.956266
BILI,0.737904,0.785894,0.763081,0.763081
PDD,0.334841,0.781684,0.667191,0.667191
NVDA,0.623663,0.525030,0.658659,0.514668
U,0.872125,0.872125,0.872125,0.872125
PTON,0.522539,0.965126,0.965126,0.965126
GOOG,0.614278,0.338201,0.303100,0.270467
AMZN,0.033131,0.231665,0.342583,0.325184
MSFT,0.337535,0.395207,0.386741,0.298489
FB,0.313541,0.324557,0.275511,0.315462


We see that the annualized returns are skewed to the right due to the broad spike in equity valuations in 2020. Also, some of the companies in the list only went public in the last 5 years, which explains the potentially transitory outperformance against the benchmark. A subtle takeaway is that some of the returns are largely driven by a particular breakout year (e.g. AMZN from March to September 2020), which serves as a timely caution against making large portfolio adjustments based on 1-year performance alone. 

### Variance

In [8]:
# compute the daily standard deviation across a 1 year, 3 year, 5 year & 10 year horizon.
df_std_1y = pd.DataFrame(data = df_close_1y_norm.std(axis = 0), columns = ['1Y Daily Standard Deviation'])  
df_std_3y = pd.DataFrame(data = df_close_3y_norm.std(axis = 0), columns = ['3Y Daily Standard Deviation']) 
df_std_5y = pd.DataFrame(data = df_close_5y_norm.std(axis = 0), columns = ['5Y Daily Standard Deviation']) 
df_std_10y = pd.DataFrame(data = df_close_10y_norm.std(axis = 0), columns = ['10Y Daily Standard Deviation']) 
data_frames = [df_std_1y, df_std_3y, df_std_5y, df_std_10y]
df_std_merge = pd.concat(data_frames, axis = 1)

# compute the daily variance across a 1 year, 3 year, 5 year & 10 year horizon.
df_var_1y = pd.DataFrame(data = df_close_1y_norm.var(axis = 0), columns = ['1Y Daily Variance'])  
df_var_3y = pd.DataFrame(data = df_close_3y_norm.var(axis = 0), columns = ['3Y Daily Variance']) 
df_var_5y = pd.DataFrame(data = df_close_5y_norm.var(axis = 0, skipna = True), columns = ['5Y Daily Variance']) 
df_var_10y = pd.DataFrame(data = df_close_10y_norm.var(axis = 0, skipna = True), columns = ['10Y Daily Variance']) 
data_frames = [df_var_1y, df_var_3y, df_var_5y, df_var_10y]
df_var_merge = pd.concat(data_frames, axis = 1)

df_var_merge

,1Y Daily Variance,3Y Daily Variance,5Y Daily Variance,10Y Daily Variance
SE,0.101417,45.043906,32.211235,32.211235
BILI,0.428310,8.803575,10.879348,10.879348
PDD,0.147547,6.576315,3.439483,3.439483
NVDA,0.042424,0.520847,9.420006,228.995407
U,0.106797,0.106797,0.106797,0.106797
PTON,0.062712,2.802343,2.802343,2.802343
GOOG,0.063946,0.156413,0.378539,3.890161
AMZN,0.002200,0.121470,1.235877,20.957742
MSFT,0.015483,0.266703,1.544241,10.308184
FB,0.015004,0.123917,0.222475,4.597913


In [9]:
# compute the pearson pairwise correlation matrix
df_return_1y_corr = df_close_1y_norm.corr(method='pearson')
df_return_3y_corr = df_close_3y_norm.corr(method='pearson')
df_return_5y_corr = df_close_5y_norm.corr(method='pearson')
df_return_10y_corr = df_close_10y_norm.corr(method='pearson')

In [10]:
# plotting the 1 year correlation matrix 

sns.set_theme(style="white")
# generate a mask for the upper triangle
mask = np.triu(np.ones_like(df_return_1y_corr, dtype=bool))

# set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# generate a custom diverging colormap
cmap = sns.diverging_palette(240, 5, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_return_1y_corr, mask=mask, cmap=cmap, vmax=1, center=0, vmin = -1,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

This result is sort of expected. In general, we know that the big tech should be highly correlated with the S&P500, and the so-called "high-beta/ARK" stocks should correlate with each other.

In [11]:
# compute the pairwise variance covariance matrix 
df_return_1y_cov = df_close_1y_norm.cov() 
df_return_3y_cov = df_close_3y_norm.cov()
df_return_5y_cov = df_close_5y_norm.cov()
df_return_10y_cov = df_close_10y_norm.cov()

### Risk-Return Measures

In [12]:
# Sharpe ratio 
df_sharpe_merge = np.divide(df_return_merge,df_std_merge)
df_sharpe_merge.rename(columns = {'1Y Annual Return':'1Y Sharpe', '3Y Annual Return':'3Y Sharpe', '5Y Annual Return':'5Y Sharpe','10Y Annual Return':'10Y Sharpe'}, inplace=True)
df_sharpe_merge

,1Y Sharpe,3Y Sharpe,5Y Sharpe,10Y Sharpe
SE,2.946512,0.182957,0.168490,0.168490
BILI,1.127511,0.264871,0.231350,0.231350
PDD,0.871713,0.304818,0.359753,0.359753
NVDA,3.027908,0.727493,0.214603,0.034011
U,2.668701,2.668701,2.668701,2.668701
PTON,2.086624,0.576532,0.576532,0.576532
GOOG,2.429182,0.855142,0.492641,0.137129
AMZN,0.706430,0.664701,0.308161,0.071033
MSFT,2.712602,0.765263,0.311216,0.092969
FB,2.559752,0.921991,0.584115,0.147118


Comparing the sharpe ratios of the individual stocks against the benchmark, it becomes apparent that beating the market on a risk-adjusted basis is no simple feat (at least when we use volatility as a risk measure). 

## 2. Portfolio Optimization

In this section, we will start off by experimenting with the PyPortfolioOpt library (Martin, 2021). His work can be accessed [here](https://joss.theoj.org/papers/10.21105/joss.03066).

In [43]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.hierarchical_portfolio import HRPOpt

### Current Allocation

In [38]:
# current weightage 
arr_w = np.array([[0.30,0.12,0.10,0.10,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01]])
df_w = pd.DataFrame(data = arr_w, columns = s_order)
df_w = df_w.rename(index={0: "weight"})
df_w

,SE,BILI,NVDA,PDD,U,FB,GOOG,AMZN,MSFT,INTC,ROKU,PTON,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT
weight,0.3,0.12,0.1,0.1,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01


In [39]:
arr_w.sum()

1.0

In [16]:
# split dataframe into portfolio and benchmark
df_return_1y_mean_port = df_return_1y_mean.iloc[0:18,:]
df_return_1y_mean_bench = df_return_1y_mean.loc['S&P500']

df_return_1y_cov_port = df_return_1y_cov.iloc[0:18,0:18]

In [17]:
# convert dataframe back to numpy array for ease of manipulation
np_return_1y_mean = df_return_1y_mean_port.T.to_numpy()

np_return_1y_cov = df_return_1y_cov_port.to_numpy()

In [18]:
# portfolio return 
portfolio_return_1y = np.inner(np_return_1y_mean,arr_w)
portfolio_var_1y = np.matmul(np.matmul(arr_w,np_return_1y_cov),arr_w.T)
portfolio_sharpe_1y = portfolio_return_1y/np.sqrt(portfolio_var_1y)
benchmark_sharpe_1y = df_sharpe_merge['1Y Sharpe']['S&P500']
print(f'Portfolio 1y return:{portfolio_return_1y}; Portfolio 1y variance:{portfolio_var_1y}')
print(f'Portfolio 1y sharpe ratio:{portfolio_sharpe_1y}; S&P500 1y Sharpe: {benchmark_sharpe_1y}')

Portfolio 1y return:[[0.67775133]]; Portfolio 1y variance:[[0.0642323]]
Portfolio 1y sharpe ratio:[[2.67419846]]; S&P500 1y Sharpe: 2.6639895476920965


Here, we see the diversification effects of a multi-asset portfolio. While the 1 year portfolio returns remains at a respectable 69%, the portfolio volatility is drastically reduced as compared to the volatility of the individual stocks. In MPT terminology, the idiosyncratic risks of individual assets is reduced.

However, we once again see the power of the S&P500, which boosts a Sharpe ratio of 2.72 compared to our current portfolio Sharpe ratio of 2.63. Now let's see if we can use the MPT techniques to construct a portfolio that outperforms the market on a risk-adjusted basis. 

### Maximum Sharpe Portfolio 

In [19]:
# split dataframe into portfolio and benchmark
df_close_1y_port = df_close_1y.iloc[:,0:18]
df_close_1y_bench = df_close_1y['S&P500']

In [42]:
# Expected returns
mu_a = expected_returns.mean_historical_return(df_close_1y_port, compounding = False)
mu_g = expected_returns.mean_historical_return(df_close_1y_port, compounding = True)
mu_ema = expected_returns.ema_historical_return(df_close_1y_port, compounding=True)
mu_capm = expected_returns.capm_return(df_close_1y_port)
data_frames = [mu_a,mu_g,mu_ema,mu_capm]
df_merge = pd.concat(data_frames, axis = 1)
df_merge = df_merge.rename(columns = {0: 'Arithmetic', 1: 'Geometric', 2:'Exponential MA', 'mkt': 'CAPM'})
df_merge

,Arithmetic,Geometric,2021-08-30 00:00:00,CAPM
SE,0.938347,1.178173,1.594172,1.082530
BILI,0.737904,0.571706,0.425175,1.210056
PDD,0.334841,0.068191,0.061350,1.193159
NVDA,0.623663,0.701934,1.170043,0.783468
U,0.872125,0.916446,1.059787,1.064249
PTON,0.522539,0.325074,0.294785,0.958415
GOOG,0.614278,0.784432,0.941141,0.386380
AMZN,0.033131,-0.008550,0.076556,0.487503
MSFT,0.337535,0.360129,0.519717,0.424872
FB,0.313541,0.299646,0.525554,0.479567


In [22]:
# Risk matrix
S = risk_models.sample_cov(df_close_1y_port)

In [23]:
# Optimize for maximum Sharpe ratio using arithmetic mean & sample covariance matrix
ef_a = EfficientFrontier(mu_a, S)
raw_weights = ef_a.max_sharpe()
cleaned_weights = ef_a.clean_weights()
# ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef_a.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.03445), ('BILI', 0.0), ('PDD', 0.0), ('NVDA', 0.0), ('U', 0.02837), ('PTON', 0.00184), ('GOOG', 0.30303), ('AMZN', 0.0), ('MSFT', 0.0), ('FB', 0.0), ('INTC', 0.0), ('ROKU', 0.04206), ('RBLX', 0.00911), ('BLK', 0.13278), ('BRK-B', 0.32203), ('ADSK', 0.0), ('DKNG', 0.0), ('CLPT', 0.12633)])
Expected annual return: 67.1%
Annual volatility: 20.5%
Sharpe Ratio: 3.18


(0.6708858427603841, 0.20479438986785636, 3.178240591357841)

Here, we see the central limitation behind the methodology of using historical returns to estimate the expected (future) returns. The allocation model placed a disproportionate weight on the outperformers in the past year (i.e. google, berkshire & clearpoint), outputing a portfolio that looks good on paper but probably does not hold its weight over time. Let's try to do better. 

In [24]:
# Optimize for maximum Sharpe ratio using arithmetic mean & sample covariance matrix
ef_capm = EfficientFrontier(mu_capm, S)
raw_weights = ef_capm.max_sharpe()
cleaned_weights = ef_capm.clean_weights()
print(cleaned_weights)
ef_capm.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.05115), ('BILI', 0.05625), ('PDD', 0.04733), ('NVDA', 0.04597), ('U', 0.0506), ('PTON', 0.05476), ('GOOG', 0.05504), ('AMZN', 0.06369), ('MSFT', 0.09861), ('FB', 0.0358), ('INTC', 0.04477), ('ROKU', 0.05222), ('RBLX', 0.05723), ('BLK', 0.05068), ('BRK-B', 0.08313), ('ADSK', 0.04594), ('DKNG', 0.05251), ('CLPT', 0.0543)])
Expected annual return: 70.2%
Annual volatility: 27.4%
Sharpe Ratio: 2.49


(0.7020164825777175, 0.2738087083985657, 2.4908502237443453)

### Black-Litterman Portfolio

In [95]:
S = risk_models.sample_cov(df_close_1y_port)
view_dict = {"SE": 0.50, "BILI": 0.42, "NVDA": 0.38, "PDD": 0.38, "U": 0.36, "FB": 0.33}
confidence_dict = np.array([0.8,0.8,0.8,0.8,0.8,0.8])
bl = BlackLittermanModel(S, pi="equal", absolute_views=view_dict, omega="idzorek", view_confidences = confidence_dict,tau=0.1)
rets = bl.bl_returns()
var = pd.DataFrame(np.diag(S), index= S.index , columns = ['Variance'], dtype = float)

# black litterman returns
df_merge = pd.concat([rets, var], axis = 1)
df_merge = df_merge.rename(columns = {0: 'Return'})
df_merge['Sharpe'] = df_merge['Return']/df_merge['Variance']
df_merge

,Return,Variance,Sharpe
SE,0.445536,0.317611,1.402774
BILI,0.402582,0.579822,0.694320
PDD,0.387624,0.557741,0.694989
NVDA,0.359296,0.182918,1.964252
U,0.351562,0.444680,0.790596
PTON,0.294903,0.477735,0.617293
GOOG,0.207297,0.069552,2.980452
AMZN,0.238835,0.083562,2.858156
MSFT,0.211816,0.059536,3.557786
FB,0.291293,0.103427,2.816413


In [41]:
ef_bl = EfficientFrontier(rets, S)
raw_weights = ef_bl.max_sharpe()
cleaned_weights = ef_bl.clean_weights()
print(cleaned_weights)
ef_bl.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.14464), ('BILI', 0.02425), ('PDD', 0.0), ('NVDA', 0.07192), ('U', 0.01914), ('PTON', 0.01491), ('GOOG', 0.0), ('AMZN', 0.04511), ('MSFT', 0.06722), ('FB', 0.32703), ('INTC', 0.0), ('ROKU', 0.00746), ('RBLX', 0.01434), ('BLK', 0.0), ('BRK-B', 0.26255), ('ADSK', 0.0), ('DKNG', 0.0), ('CLPT', 0.00143)])
Expected annual return: 25.6%
Annual volatility: 23.0%
Sharpe Ratio: 1.03


(0.2564976963940001, 0.22960913796198798, 1.0300012381613164)

We see that the model favors stocks with high sharpe-ratio(e.g. FB) and negative correlation with other assets(e.g. BRK-B). 

Now let's try out recent developments in the field like shrinkage and Hierarchical Risk Parity (López de Prado, 2016), along with some novel experimental features like exponentially-weighted covariance matrices.

hrp = HRPOpt()


## 3.Extensions

As per PyPortfolioOpt's roadmap, I aim to include conditional drawdown optimization (Chekhlov et al., 2005), risk parity portfolios (Spinu,2013), higher moment optimization (Harvey et al., 2010), and factor models into my analysis in the near future. Stay tuned!